In [5]:
#!pip install tensorflow==1.14.0

In [6]:
#!pip install keras==2.2.5

In [7]:
import os
os.environ["PYTHONHASHSEED"] = "13"
os.environ["CUDA_VISIBLE_DEVICES"] = "0:1"

import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(13)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(13)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(13)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [8]:
import gc
import warnings

#import lightgbm as lgb
import pandas as pd

from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from tqdm.notebook import tqdm

import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Dense, Dropout

path_data = "../input/ashrae-energy-prediction/"
path_train = path_data + "train.csv"
path_test = path_data + "test.csv"
path_building = path_data + "building_metadata.csv"
path_weather_train = path_data + "weather_train.csv"
path_weather_test = path_data + "weather_test.csv"

myfavouritenumber = 13
seed = myfavouritenumber

warnings.filterwarnings("ignore")

In [9]:
site_id = 5

In [10]:
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

building = pd.read_csv(path_building)

weather_train = pd.read_csv(path_weather_train)
weather_test = pd.read_csv(path_weather_test)

weather_train.drop(["sea_level_pressure", "wind_direction", "wind_speed"], axis=1, inplace=True)
weather_test.drop(["sea_level_pressure", "wind_direction", "wind_speed"], axis=1, inplace=True)

weather_train = weather_train.groupby("site_id").apply(lambda group: group.interpolate(limit_direction="both"))
weather_test = weather_test.groupby("site_id").apply(lambda group: group.interpolate(limit_direction="both"))

df_train = df_train.merge(building, on="building_id")
df_test = df_test.merge(building, on="building_id")

df_train = df_train[df_train.site_id == site_id]
df_test = df_test[df_test.site_id == site_id]

df_train["log_meter_reading"] = np.log1p(df_train.meter_reading)

df_train = df_train[~((df_train.building_id == 656) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 657) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 658) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 659) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 660) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 661) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 662) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 663) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 664) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 666) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 667) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 668) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 669) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 669) & (df_train.log_meter_reading > 3))]
df_train = df_train[~((df_train.building_id == 670) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 671) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 672) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 674) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 675) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 675) & (df_train.timestamp >= "2016-02") & (df_train.timestamp < "2016-06"))]
df_train = df_train[~((df_train.building_id == 677) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 678) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 680) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 681) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 683) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 684) & (df_train.log_meter_reading < 1.5))]
df_train = df_train[~((df_train.building_id == 685) & (df_train.log_meter_reading < 2))]
df_train = df_train[~((df_train.building_id == 687) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 688) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 689) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 690) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 692) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 693) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 694) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 697) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 698) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 701) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 702) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 703) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 704) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 705) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 707) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 708) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 709) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 710) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 711) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 712) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 713) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 714) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 715) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 716) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 717) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 718) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 719) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 720) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 721) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 722) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 723) & (df_train.log_meter_reading < 2))]
df_train = df_train[~((df_train.building_id == 724) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 725) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 727) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 728) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 729) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 730) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 731) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 732) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 733) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 734) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 735) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 736) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 739) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 741) & (df_train.log_meter_reading < 0.5))]
df_train = df_train[~((df_train.building_id == 742) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 743) & (df_train.log_meter_reading < 1))]
df_train = df_train[~((df_train.building_id == 744) & (df_train.log_meter_reading < 1))]
df_train = df_train[df_train.meter == 0]

df_train = df_train.merge(weather_train, on=["site_id", "timestamp"], how="left")
df_test = df_test.merge(weather_test, on=["site_id", "timestamp"], how="left")

df_train.timestamp = pd.to_datetime(df_train.timestamp, format='%Y-%m-%d %H:%M:%S')
df_test.timestamp = pd.to_datetime(df_test.timestamp, format='%Y-%m-%d %H:%M:%S')

df_train.building_id = df_train.building_id.astype("str")
df_test.building_id = df_test.building_id.astype("str")

del building
gc.collect()

35

In [11]:
df_train["hour"] = df_train.timestamp.dt.hour.astype("str")
df_test["hour"] = df_test.timestamp.dt.hour.astype("str")

df_train["weekday"] = df_train.timestamp.dt.weekday.astype("str")
df_test["weekday"] = df_test.timestamp.dt.weekday.astype("str")

df_train["month"] = df_train.timestamp.dt.month.astype("str")
df_test["month"] = df_test.timestamp.dt.month.astype("str")

In [12]:
features = [
    "building_id",
    "square_feet",
    "year_built",
    "floor_count",
    "primary_use",
    "air_temperature",
    "cloud_coverage",
    "dew_temperature",
    "precip_depth_1_hr",
    "hour",
    "weekday",
    "month"
]

In [13]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [14]:
cv = 4
folds = KFold(n_splits = cv, shuffle = False, random_state = seed)

df_cv = []
df_preds = []

Building Model for site_id 5 and meter 0
Preparing Site: 5 , meter: 0 , Fold: 1



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 569823 samples, validate on 189942 samples
Epoch 1/31




 - 21s - loss: 1.9184 - val_loss: 1.0885
Epoch 2/31
 - 3s - loss: 1.2061 - val_loss: 1.0609
Epoch 3/31
 - 3s - loss: 1.1680 - val_loss: 1.0736
Epoch 4/31
 - 3s - loss: 1.1515 - val_loss: 1.0925
Epoch 5/31
 - 3s - loss: 1.1376 - val_loss: 1.1126
Epoch 6/31
 - 3s - loss: 1.1253 - val_loss: 1.1257
Epoch 7/31
 - 3s - loss: 1.1139 - val_loss: 1.1212
Epoch 8/31
 - 3s - loss: 1.1047 - val_loss: 1.1268
Epoch 9/31
 - 3s - loss: 1.0957 - val_loss: 1.1252
Epoch 10/31
 - 3s - loss: 1.0873 - val_loss: 1.1165
Epoch 11/31
 - 3s - loss: 1.0805 - val_loss: 1.1134
Epoch 12/31
 - 3s - loss: 1.0733 - val_loss: 1.1129
Epoch 13/31
 - 3s - loss: 1.0683 

CPU times: user 41.3 s, sys: 2.75 s, total: 44 s
Wall time: 33.5 s
Preparing Site: 5 , meter: 0 , Fold: 2
Train on 569824 samples, validate on 189941 samples
Epoch 1/31
 - 4s - loss: 1.8910 - val_loss: 1.1114
Epoch 2/31
 - 3s - loss: 1.2158 - val_loss: 1.0508
Epoch 3/31
 - 3s - loss: 1.1696 - val_loss: 1.0231
Epoch 4/31
 - 3s - loss: 1.1481 - val_loss: 1.0143
Epoch 5/31
 - 3s - loss: 1.1300 - val_loss: 1.0085
Epoch 6/31
 - 3s - loss: 1.1143 - val_loss: 0.9856
Epoch 7/31
 - 3s - loss: 1.1015 - val_loss: 0.9686
Epoch 8/31
 - 3s - loss: 1.0898 - val_loss: 0.9573
Epoch 9/31
 - 3s - loss: 1.0799 - val_loss: 0.9670
Epoch 10/31
 - 3s - loss: 1.0716 - val_loss: 0.9662
Epoch 11/31
 - 3s - loss: 1.0634 - val_loss: 0.9656
Epoch 12/31
 - 3s - loss: 1.0560 - val_loss: 0.9624
Epoch 13/31
 - 3s - loss: 1.0495 - val_loss: 0.9606
Epoch 14/31
 - 3s - loss: 1.0432 - val_loss: 0.9663
Epoch 15/31
 - 3s - loss: 1.0379 - val_loss: 0.9666
Epoch 16/31
 - 3s - loss: 1.0329 - val_loss: 0.9717
Epoch 17/31
 - 3s -

CPU times: user 42.4 s, sys: 3.48 s, total: 45.9 s
Wall time: 35.4 s
Preparing Site: 5 , meter: 0 , Fold: 3
Train on 569824 samples, validate on 189941 samples
Epoch 1/31
 - 5s - loss: 2.0447 - val_loss: 0.7137
Epoch 2/31
 - 3s - loss: 1.3481 - val_loss: 0.6882
Epoch 3/31
 - 3s - loss: 1.2952 - val_loss: 0.6856
Epoch 4/31
 - 3s - loss: 1.2790 - val_loss: 0.6634
Epoch 5/31
 - 3s - loss: 1.2647 - val_loss: 0.6554
Epoch 6/31
 - 3s - loss: 1.2486 - val_loss: 0.6330
Epoch 7/31
 - 3s - loss: 1.2350 - val_loss: 0.6170
Epoch 8/31
 - 3s - loss: 1.2237 - val_loss: 0.6142
Epoch 9/31
 - 3s - loss: 1.2142 - val_loss: 0.5994
Epoch 10/31
 - 3s - loss: 1.2059 - val_loss: 0.5976
Epoch 11/31
 - 3s - loss: 1.1988 - val_loss: 0.5941
Epoch 12/31
 - 3s - loss: 1.1924 - val_loss: 0.5842
Epoch 13/31
 - 3s - loss: 1.1859 - val_loss: 0.5853
Epoch 14/31
 - 3s - loss: 1.1812 - val_loss: 0.5849
Epoch 15/31
 - 3s - loss: 1.1757 - val_loss: 0.5879
Epoch 16/31
 - 3s - loss: 1.1706 - val_loss: 0.5861
Epoch 17/31
 - 3s

CPU times: user 44.2 s, sys: 2.71 s, total: 46.9 s
Wall time: 36.4 s
Preparing Site: 5 , meter: 0 , Fold: 4
Train on 569824 samples, validate on 189941 samples
Epoch 1/31
 - 5s - loss: 1.8454 - val_loss: 1.3850
Epoch 2/31
 - 3s - loss: 1.1601 - val_loss: 1.2769
Epoch 3/31
 - 3s - loss: 1.1136 - val_loss: 1.2536
Epoch 4/31
 - 3s - loss: 1.0953 - val_loss: 1.2349
Epoch 5/31
 - 3s - loss: 1.0781 - val_loss: 1.1994
Epoch 6/31
 - 3s - loss: 1.0634 - val_loss: 1.1684
Epoch 7/31
 - 3s - loss: 1.0494 - val_loss: 1.1662
Epoch 8/31
 - 3s - loss: 1.0382 - val_loss: 1.1494
Epoch 9/31
 - 3s - loss: 1.0276 - val_loss: 1.1461
Epoch 10/31
 - 3s - loss: 1.0185 - val_loss: 1.1210
Epoch 11/31
 - 3s - loss: 1.0114 - val_loss: 1.1181
Epoch 12/31
 - 3s - loss: 1.0037 - val_loss: 1.1061
Epoch 13/31
 - 3s - loss: 0.9980 - val_loss: 1.1137
Epoch 14/31
 - 3s - loss: 0.9930 - val_loss: 1.0995
Epoch 15/31
 - 3s - loss: 0.9873 - val_loss: 1.1105
Epoch 16/31
 - 3s - loss: 0.9823 - val_loss: 1.0917
Epoch 17/31
 - 3s

CPU times: user 45.8 s, sys: 2.7 s, total: 48.5 s
Wall time: 37.9 s

CV RMSE for Site: 5 and Meter: 0 is 1.0801081981510963

CV RMSE for Site: 5 is 1.0801081981510963


In [15]:
for meter in df_train.meter.unique():
    print("Building Model for site_id", site_id, "and meter", meter)
        
    df_train_site_meter = df_train[df_train.meter == meter].reset_index(drop=True)
    df_test_site_meter = df_test[df_test.meter == meter].reset_index(drop=True)
    
    pred_val = np.zeros(df_train_site_meter.shape[0])
    pred_test = np.zeros(df_test_site_meter.shape[0])

    row_ids_test = df_test_site_meter.row_id
    y_train_site_meter = df_train_site_meter.log_meter_reading.values
    
    df_train_site_meter = df_train_site_meter[features]
    df_test_site_meter = df_test_site_meter[features]

    df_train_site_meter = pd.get_dummies(df_train_site_meter)
    df_test_site_meter = pd.get_dummies(df_test_site_meter)
        
    for k, (train_idx, valid_idx) in enumerate(folds.split(df_train_site_meter, y_train_site_meter)):
        print("Preparing Site:", site_id, ", meter:", meter, ", Fold:", k+1)
        
        X_train, y_train = df_train_site_meter.iloc[train_idx], y_train_site_meter[train_idx]
        X_valid, y_valid = df_train_site_meter.iloc[valid_idx], y_train_site_meter[valid_idx]
        
        X_train.dropna(axis=1, how="all", inplace=True)
        X_train = X_train.loc[:, (X_train != X_train.iloc[0]).any()]

        X_valid = X_valid[X_train.columns]
        X_test = df_test_site_meter[X_train.columns]
        
        df_means = pd.concat([X_train, X_valid, X_test], ignore_index=True).mean()
        X_train.fillna(df_means, inplace=True)
        X_valid.fillna(df_means, inplace=True)
        X_test.fillna(df_means, inplace=True)
        
        sc = MinMaxScaler()
        sc.fit(pd.concat([X_train, X_valid, X_test], ignore_index=True))
        
        X_train = np.array(sc.transform(X_train))
        X_valid = np.array(sc.transform(X_valid))
        X_test = np.array(sc.transform(X_test))
        
        model = Sequential()
        model.add(Dense(128, input_dim=X_train.shape[1], activation="relu", kernel_initializer=keras.initializers.glorot_uniform(seed=seed)))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        model.add(Dense(32, activation="tanh", kernel_initializer=keras.initializers.glorot_uniform(seed=seed)))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        model.add(Dense(4, activation="tanh", kernel_initializer=keras.initializers.glorot_uniform(seed=seed)))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        model.add(Dense(1))
        model.compile(loss=rmse, optimizer="adam")
        
        model_ffnn = model.fit(X_train, y_train, epochs=31, batch_size=1000, validation_data=(X_valid, y_valid), verbose=2, shuffle=False)

        plt.plot(model_ffnn.history["loss"])
        plt.plot(model_ffnn.history["val_loss"])
        plt.title("Loss for Site: " + str(site_id) + ", Meter: " + str(meter) + ", Fold: " + str(k+1))
        plt.ylabel("rmse")
        plt.xlabel("Epoch")
        plt.legend(["train", "valid"], loc="upper right")
        plt.show()
        
        pred_val[valid_idx] = model.predict(X_valid).ravel()
        %time pred_test += model.predict(X_test).ravel() / cv
        
        gc.collect()
        
    df_valid = pd.DataFrame({"site_id": site_id, "meter": meter, "meter_reading": np.expm1(y_train_site_meter), "pred_ffnn": np.clip(np.expm1(pred_val), 0, a_max=None)})
    df_pred = pd.DataFrame({"row_id": row_ids_test, "meter_reading": np.clip(np.expm1(pred_test), 0, a_max=None)})
    
    df_cv.append(df_valid)
    df_preds.append(df_pred)
    
    gc.collect()
    
    print("\nCV RMSE for Site:", site_id, "and Meter:", meter, "is", np.sqrt(mean_squared_error(np.log1p(df_valid.meter_reading), np.log1p(df_valid.pred_ffnn))))

df_cv = pd.concat(df_cv)
print("\nCV RMSE for Site:", site_id, "is", np.sqrt(mean_squared_error(np.log1p(df_cv.meter_reading), np.log1p(df_cv.pred_ffnn))))


In [17]:
df_preds = pd.concat(df_preds)
df_preds.to_csv("../sub/ffnn_pred_site_" + str(site_id) + ".csv", index=False)